# 3 TESTING NOTEBOOK - TEST 2

In [1]:
# INSTALL SCIKERAS PACKAGES
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
# IMPORT TENSORFLOW/KERAS
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.preprocessing import image
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

from keras.layers.core import Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

# SCIKIT-LEARN/SCIKERAS
from sklearn import preprocessing
import sklearn.metrics as metrics
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import classification_report

# IMPORT OTHER MODULES
import os
import os.path
import sys
import glob
import datetime
from pathlib import Path

import fnmatch # to count number of image per folder easily
import random # to select random images from a folder
import numpy as np
import pandas as pd
from os import mkdir
import seaborn as sns
from pathlib import Path
from shutil import copyfile # to import "copyfile"
from numpy.random import seed
import matplotlib.image as img_mat
import plotly.figure_factory as ff # for printing the heatmap
from collections import defaultdict
from matplotlib import pyplot as plt
from IPython.display import Javascript

# CV2
import cv2
from google.colab.patches import cv2_imshow 

In [3]:
# Link Google Drive account
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# CONSTANT
SEED_VALUE = 42
num_classes = 9 # number of output classes
batch_size = 16
SIZE = 224  #Resize images --> https://www.researchgate.net/post/Which_Image_resolution_should_I_use_for_training_for_deep_neural_network#:~:text=So%20the%20rule%20of%20thumb,for%20something%20smaller%20and%20easier.

# PATH TO THE DATA
ORIGINAL_DATA = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset"
base_path_test = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/test/'
DIR_TEST_DATA = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/test/*' # path contenente i sottoinsiemi di train, test

In [5]:
# # # Capture training data and labels into respective lists
# # train_images = []
# # train_labels = [] 

# # # GET THE LABELS
# # for directory_path in glob.glob(DIR_TRAIN_DATA):
# #     tr_label = directory_path.split("/")[-1]
# #     print(tr_label)
# #     # GET THE IMAGES
# #     for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
# #         # print(img_path)
# #         img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
# #         img = cv2.resize(img, (SIZE, SIZE))
# #         img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
# #         train_images.append(img)
# #         train_labels.append(tr_label)

# # # CONVERT LISTS TO ARRAYS
# # train_images = np.array(train_images)
# # train_labels = np.array(train_labels)

# ################################################################
# # CAPTURE TEST/VALIDATION DATA AND LABELS INTO RESPECTIVE LISTS

# test_images = []
# test_labels = [] 
# for directory_path in glob.glob(DIR_TEST_DATA):
#     tt_label = directory_path.split("/")[-1]
#     for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
#         img = cv2.imread(img_path, cv2.IMREAD_COLOR)
#         img = cv2.resize(img, (SIZE, SIZE))
#         img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
#         test_images.append(img)
#         test_labels.append(tt_label)

# #Convert lists to arrays                
# test_images = np.array(test_images)
# test_labels = np.array(test_labels)

In [6]:
# def prep_fn(img):
#     img = img.astype(np.float32) / 255.0
#     img = (img - 0.5) * 2
#     return img

In [7]:
preprocess_input_mobilenetV2 = tf.keras.applications.mobilenet_v2.preprocess_input

In [8]:
base_path_train = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/'

# train_datagen_mnetv2 = ImageDataGenerator(preprocessing_function = preprocess_input_mobilenetV2)
train_datagen_mnetv2 = ImageDataGenerator()

train_generator_mnetv2 = train_datagen_mnetv2.flow_from_directory(
    base_path_train,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=16,
    shuffle=True,
    class_mode="categorical",
    seed = SEED_VALUE)

Found 1803 images belonging to 9 classes.


In [9]:
# test_datagen_mnetv2 = ImageDataGenerator() # rescale=1./255
# test_datagen_mnetv2 = ImageDataGenerator(preprocessing_function = preprocess_input_mobilenetV2) 
test_datagen_mnetv2 = ImageDataGenerator()

# TOGLIERE: USARE SOLO NEL NOTEBOOK DI TEST!
test_generator = test_datagen_mnetv2.flow_from_directory(
    base_path_test,
    target_size=(SIZE, SIZE), # target_size=(224,224),
    color_mode="rgb",
    batch_size=1,
    # batch_size=batch_size,
    shuffle=False,
    # class_mode="categorical",
    class_mode=None,
    seed = SEED_VALUE)

Found 155 images belonging to 9 classes.


In [10]:
ground_truth_dict = test_generator.class_indices
ground_truth_dict

{'caesar_salad': 0,
 'caprese_salad': 1,
 'french_fries': 2,
 'greek_salad': 3,
 'hamburger': 4,
 'hot_dog': 5,
 'pizza': 6,
 'sashimi': 7,
 'sushi': 8}

In [11]:
# LOAD SAVED MODEL AND GET PATH FOR AN IMAGE
best_model = tf.keras.models.load_model("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/saved_models/model_mobilenetV2.h5") 

In [12]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

# test_generator.reset() # https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

pred = best_model.predict(test_generator, 
                          steps=STEP_SIZE_TEST)

155/155 [==============================] - 23s 138ms/step


In [13]:
len(pred)

155

In [14]:
predicted_class_indices=np.argmax(pred,axis=1)

In [15]:
predicted_class_indices

array([2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 3, 4, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2,
       2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2,
       3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 3, 4, 2, 2, 2, 2, 4, 2, 2, 3, 3,
       2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 3,
       4])

In [16]:
labels = (train_generator_mnetv2.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [17]:
test_generator.filenames

['caesar_salad/crop_caesar1.jpg_RESIZED.jpg',
 'caesar_salad/crop_caesar66.jpg_RESIZED.jpg',
 'caesar_salad/smaller_salad (100).jpg_RESIZED.jpg',
 'caesar_salad/smaller_salad (101).jpg_RESIZED.jpg',
 'caesar_salad/smaller_salad (7).jpg_RESIZED.jpg',
 'caprese_salad/crop_caesar7.jpg_RESIZED.jpg',
 'caprese_salad/smaller_salad (11).jpg_RESIZED.jpg',
 'caprese_salad/smaller_salad (20).jpg_RESIZED.jpg',
 'french_fries/cropfries13.jpg_RESIZED.jpg',
 'french_fries/cropfries14.jpg_RESIZED.jpg',
 'french_fries/cropfries15.jpg_RESIZED.jpg',
 'french_fries/cropfries28.jpg_RESIZED.jpg',
 'french_fries/cropfries29.jpg_RESIZED.jpg',
 'french_fries/cropfries31.jpg_RESIZED.jpg',
 'french_fries/cropfries33.jpg_RESIZED.jpg',
 'french_fries/cropfries4.jpg_RESIZED.jpg',
 'french_fries/cropfries53.jpg_RESIZED.jpg',
 'french_fries/cropfries58.jpg_RESIZED.jpg',
 'french_fries/cropfries59.jpg_RESIZED.jpg',
 'french_fries/cropfries6.jpg_RESIZED.jpg',
 'french_fries/cropfries61.jpg_RESIZED.jpg',
 'french_fries

In [18]:
filenames=test_generator.filenames
# filenames=train_generator_mnetv2.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

-------------------

In [35]:
# use Path object to grab all the images in a folder
caesar_salad_dir  = Path("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset/caesar_salad")
caprese_salad_dir = Path("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset/caprese_salad")
french_fries_dir  = Path("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset/french_fries")
greek_salad_dir   = Path("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset/greek_salad")
hamburger_dir     = Path("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset/hamburger")
hot_dog_dir       = Path("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset/hot_dog")
pizza_dir         = Path("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset/pizza")
sashimi_dir       = Path("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset/sashimi")
sushi_dir         = Path("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset/sushi")

In [36]:
def generate_df(image_dir, label): 
  # get all the filepath. Convert to string for using flow_from_dataframe later
  filepaths = pd.Series(list(image_dir.glob(r'*.jpg')), name="Filepath").astype(str) # name: give the Series a name
  # labels
  labels = pd.Series(label, name="Label", index=filepaths.index)
  # concatenate two series as a dataframe
  df = pd.concat([filepaths, labels], axis=1)

  return df

In [37]:
caesar_salad_df  = generate_df(caesar_salad_dir,  label="CAESAR_SALAD")
caprese_salad_df = generate_df(caprese_salad_dir, label="CAPRESE_SALAD")
french_fries_df  = generate_df(french_fries_dir,  label="FRENCH_FRIES")
greek_salad_df   = generate_df(greek_salad_dir,   label="GREEK_SALAD")
hamburger_df     = generate_df(hamburger_dir,     label="HAMBURGER")
hot_dog_df       = generate_df(hot_dog_dir,       label="HOT_DOG")
pizza_df         = generate_df(pizza_dir,         label="PIZZA")
sashimi_df       = generate_df(sashimi_dir,       label="SASHIMI")
caesar_salad_df  = generate_df(caesar_salad_dir,  label="CAESAR_SALAD")
sushi_df         = generate_df(sushi_dir,         label="SUSHI")

<ipython-input-36-3ba43c3058a0>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  filepaths = pd.Series(list(image_dir.glob(r'*.jpg')), name="Filepath").astype(str) # name: give the Series a name


In [38]:
all_df = pd.concat([caesar_salad_df,
                    caprese_salad_df,
                    french_fries_df,
                    greek_salad_df,
                    hamburger_df,
                    hot_dog_df,
                    pizza_df,
                    sashimi_df,
                    caesar_salad_df,
                    sushi_df], 
                   axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True) 
                   # sample and shuffle at the same time and reset index to have the correct values.
                   # drop the previous index so you do not have another column for that.

# problems:
#   - indeces are wrong
#   - the label are on top of each other and not shuffled
all_df

,Filepath,Label


In [ ]:
# this helps us to deal with memory issues to train a large dataset. Loading and training and recycling the memory at
# the same time. You can add a Data Augmentation process but it is not mandatory.
# But we can rescale the images so the pixel data are coming in as values between 0 and 1.
# The standard encoding for pixels values are in RGB scale between 0 and 255 (255 maximum color intensity for each of the three channels).
# By dividing by the maximum value (or mutiplying by 1/255) we are scaling all the pixels value into a range between 0 and 1.
# We also include a validation split: we can only split once for a given directory and we split the
# train set into train and validation and we use a separate generator for the test set. 
train_gen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3
)

test_gen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255 # we do not include a validation split as mentioned before.
)

In [ ]:
# now we flow from the dataframes
train_data = train_gen.flow_from_dataframe(
    train_df, # this is the dataframe were we are coming from
    x_col="Filepath",# which column used as a filepath and which column is used as a label
    y_col="Label",
    target_size=(120, 120),
    color_mode="rgb", # three color channels
    class_mode="categorical", # in the case of multi-label classification the class_mode should be categorical (which is the default value). "binary" in case we have "Positive", "Negative" class for example.
    batch_size=32, # standard
    shuffle=True,
    seed=42,
    # resize the images as you want. TO DO: NEED TO CHECK WHAT WAS THE ORIGINAL SIZES OF THE IMAGES!!!
    # CHIEDERE AL PROF UNA VOLTA RIASCOLTATE TUTTE LE LEZIONI COME PROCEDERE! LE IMMAGINI MI SEMBRAVA AVESSERO
    # DIFFERENTI SIZES. DEVI PRIMA ESPLORARE IL DATASET PER POTERLO CAPIRE VERAMENTE!
    # IMMAGINO DI DOVER USARE PRIMA OPENCV ALL'INIZIO PER DESCRIVERE IL DATASET CORRETTAMENTE.
    # PASSAGGIO FONDAMENTALE DA FARE E NON DIMENTICARE COME FATTO NELLA TESI!!!
    subset="training" # when we pull from the train_gen and we specify "training" we get 0.8 reserved for training and if we specify "validation" we get the 0.2 percent 
)

valid_data = train_gen.flow_from_dataframe(
    train_df, # this is the dataframe were we are coming from
    x_col="Filepath",# which column used as a filepath and which column is used as a label
    y_col="Label",
    target_size=(120, 120),
    color_mode="rgb", # three color channels
    class_mode="categorical", # in the case of multi-label classification the class_mode should be categorical (which is the default value). "binary" in case we have "Positive", "Negative" class for example.
    batch_size=32, # standard
    shuffle=True,
    seed=42,
    # resize the images as you want. TO DO: NEED TO CHECK WHAT WAS THE ORIGINAL SIZES OF THE IMAGES!!!
    # CHIEDERE AL PROF UNA VOLTA RIASCOLTATE TUTTE LE LEZIONI COME PROCEDERE! LE IMMAGINI MI SEMBRAVA AVESSERO
    # DIFFERENTI SIZES. DEVI PRIMA ESPLORARE IL DATASET PER POTERLO CAPIRE VERAMENTE!
    # IMMAGINO DI DOVER USARE PRIMA OPENCV ALL'INIZIO PER DESCRIVERE IL DATASET CORRETTAMENTE.
    # PASSAGGIO FONDAMENTALE DA FARE E NON DIMENTICARE COME FATTO NELLA TESI!!!
    subset="validation"
)

# we do not have to specify a subset for the test data
test_data  = test_gen.flow_from_dataframe(
    test_df,
    x_col="Filepath",# which column used as a filepath and which column is used as a label
    y_col="Label",
    target_size=(120, 120),
    color_mode="rgb", # three color channels
    class_mode="categorical", # in the case of multi-label classification the class_mode should be categorical (which is the default value). "binary" in case we have "Positive", "Negative" class for example.
    batch_size=32, # standard
    shuffle=False, # we do not want to shuffle our test data, we encounter some problem otherwise evaluating the model. We want to mantain the model of the test sample because we want to compare them to the label in the end.
    seed=42,
    # resize the images as you want. TO DO: NEED TO CHECK WHAT WAS THE ORIGINAL SIZES OF THE IMAGES!!!
    # CHIEDERE AL PROF UNA VOLTA RIASCOLTATE TUTTE LE LEZIONI COME PROCEDERE! LE IMMAGINI MI SEMBRAVA AVESSERO
    # DIFFERENTI SIZES. DEVI PRIMA ESPLORARE IL DATASET PER POTERLO CAPIRE VERAMENTE!
    # IMMAGINO DI DOVER USARE PRIMA OPENCV ALL'INIZIO PER DESCRIVERE IL DATASET CORRETTAMENTE.
    # PASSAGGIO FONDAMENTALE DA FARE E NON DIMENTICARE COME FATTO NELLA TESI!!!
)

# The function goes through all the images and will count them as well.
# First dataset:  train set
# Second dataset: validation set
# Third dataset:  test set

In [ ]:
-----------------------

In [19]:
def prepare_image(file_check):
  img = image.load_img(file_check, target_size=(224, 224))
  img_array = image.img_to_array(img)
  img_array_expanded_dims = np.expand_dims(img_array, axis=0)
  return tf.keras.applications.mobilenet_v2.preprocess_input(img_array_expanded_dims)

  # preprocess_input_mobilenetV2 = tf.keras.applications.mobilenet_v2.preprocess_input

In [20]:
result = list()

imagepath1 = "/content/crop_caesar1.jpg_RESIZED.jpg"
imagepath2 = "/content/cropburger3.jpg_RESIZED.jpg"
imagepath3 = "/content/cropfries4.jpg_RESIZED.jpg"

preprocessed_image1 = prepare_image(imagepath1)
preprocessed_image2 = prepare_image(imagepath2)
preprocessed_image3 = prepare_image(imagepath3)

predictions1 = best_model.predict(preprocessed_image1)
print("pred1", predictions1[0])

predictions2 = best_model.predict(preprocessed_image2)
print("pred2", predictions2[0])

predictions3 = best_model.predict(preprocessed_image3)
print("pred3", predictions3[0])

result.append(np.argmax(predictions1))
result.append(np.argmax(predictions2))
result.append(np.argmax(predictions3))

FileNotFoundError: ignored

In [ ]:
result # french fries, sushi, sushi

In [ ]:
train_generator_mnetv2.class_indices

# Possible solutions

- https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

- https://github.com/keras-team/keras/issues/14303

In [ ]:
llllllllllllllllllllllllllllllll

In [ ]:
# from tensorflow.keras.applications.resnet50 import preprocess_input
# from keras.applications.mobilenet_v2 import preprocess_input
# from keras.applications.vgg16 import preprocess_input

def prepare_image(file_input, show=True):
    img = image.load_img(file_input, target_size=(SIZE, SIZE))
    img_array = image.img_to_array(img)
    # img_array_expanded_dims = preprocess_input(img_array)
    # img_array= (img_array/255.0)
    # img_array = img_array.astype(np.float32) / 255.0
    # img_array = (img_array - 0.5) * 2
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    # img_array_expanded_dims = preprocess_input(img_array_expanded_dims)

    if show:
      plt.imshow(img_array_expanded_dims[0])                           
      plt.axis('off')
      plt.show()

    return img_array_expanded_dims

    # return img_array_expanded_dims

In [ ]:
# CREATE A LIST TO STORE THE PATH OF EACH RANDOM IMAGE
inference_data = list()

# current_directory = Path("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset")
# current_directory = Path('/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/test/')
current_directory = Path(base_path_test)
# current_directory = Path('/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDatasetPreprocessed')

for folder in current_directory.iterdir():
  split_str = str(folder).split("/")[-1] # get the folder name
  print(split_str)
  file_path_type = [f"{current_directory}/{split_str}/*.jpg"]  
  # print(file_path_type)
  images = glob.glob(random.choice(file_path_type))
  # print(images)

  for i in range(1):
    # if len(inference_data) > 1:
    #   break
    random_image = random.choice(images)
    # print(random_image)
    inference_data.append(random_image)
    # img_raw = cv2.imread(random_image)
    print(inference_data)
    print("------------")

In [ ]:
# LOAD SAVED MODEL AND GET PATH FOR AN IMAGE
best_model = tf.keras.models.load_model("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/saved_models/model_mobilenetV2.h5") 

In [ ]:
# im_test = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/test/hot_dog/20160711_1214411.jpg_RESIZED.jpg'

In [ ]:
# img = image.load_img(im_test, target_size=(224,224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)
# print(full_model.predict(x))
# plt.imshow(img)

In [ ]:
pred_output_list = list()
ground_truth_list = list()
ground_truth_dict = test_generator.class_indices

for image_path in inference_data:
  split_str = str(image_path).split("/")[-2] # get the folder name
  print(split_str)
  new_image = prepare_image(image_path)
  # print(new_image.shape)
  pred = best_model.predict(new_image)

  # Generate arg maxes for predictions
  # output_index = np.argmax(pred[[0]], axis = 1)
  output_index = np.argmax(pred, axis = 1)
  print("Output index is:", output_index)

  pred_output_list.append(output_index[0])
  ground_truth_list.append(ground_truth_dict[f'{split_str}'])

  # if max(pred[0]) < 0.50:
  #   print("I'm sorry, I cannot recognise any type of food here! Try again!")
  #   print("The groud truth is", split_str)
  # else:
  #   value = [i for i in test_generator.class_indices if test_generator.class_indices[i]==np.argmax(pred, axis = 1)]
  #   print(f"The food in this image should be: {value[0]}.")
  #   print("The groud truth is", split_str, ".")
  
  print("")
  value = [i for i in test_generator.class_indices if test_generator.class_indices[i] == output_index]
  print(f"The food predicted is: {value[0]}.")
  print("------------------------ NEXT IMAGE ----------------------------")
  print("")

print("Predicted:", pred_output_list)
print("Ground Truth", ground_truth_list)

In [ ]:
# CREATE A DATAFRAME TO COMPARE THE RESULTS
df_comparison = pd.DataFrame({"predicted": pred_output_list, "ground truth": ground_truth_list})

df_comparison['comparison'] = np.where(df_comparison['predicted'] == df_comparison['ground truth'], 1, 
                                      np.where(df_comparison['predicted'] !=  df_comparison['ground truth'], 0, -1))

df_comparison

**MIGLIOR RISULTATO FINORA. POTREBBE ESSERE UN PROBLEMA DI INDICI???**

In [ ]:
# COMPUTE ACCURACY OVER THE TEN ELEMENTS
accuracy_over_test = (np.sum(df_comparison['comparison'])/len(df_comparison))*100
print(f"Accuracy over {len(df_comparison)} images is:", accuracy_over_test, "%")

**SPECIFICARE CHE I BAD RESULTS SONO DATI DA SUSHI E SASHIMI**

QUELLO CHE POTRESTI FARE E' PROVARE CON UN ALTRO DATASET FOOD E RIFARE LA STESSA COSA CON UN DATASET DIVERSO E MAGARE USARNE UN ALTRO ANCHE!